In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Import Dataset

In [3]:
df = pd.read_csv("/content/gdrive/My Drive/MADT8101/week2/supermarket.csv")

In [4]:
df

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,...,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION
0,200734,20071019,6,8,1,9.50,PRD0901335,CL00058,DEP00016,G00006,...,XX,OT,994108000679877,M,MM,Small Shop,Fresh,STORE00001,LS,E02
1,200805,20080328,6,21,1,1.49,PRD0901120,CL00162,DEP00055,G00016,...,XX,NaN,994110300629787,S,MM,Small Shop,Fresh,STORE00001,LS,E02
2,200716,20070612,3,9,1,0.86,PRD0900963,CL00248,DEP00089,G00030,...,NaN,NaN,994106200026009,S,MM,Small Shop,Grocery,STORE00001,LS,E02
3,200815,20080604,4,10,3,0.90,PRD0901055,CL00248,DEP00089,G00030,...,NaN,NaN,994111300076346,L,MM,Top Up,Grocery,STORE00001,LS,E02
4,200626,20060824,5,16,1,1.77,PRD0901877,CL00169,DEP00056,G00017,...,NaN,NaN,994102000065854,L,LA,Top Up,Fresh,STORE00001,LS,E02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578077,200738,20071116,6,12,1,0.01,PRD0903678,CL00222,DEP00076,G00023,...,UM,NaN,994108400460274,S,MM,Small Shop,Grocery,STORE00001,LS,E02
578078,200716,20070612,3,12,1,13.76,PRD0902312,CL00230,DEP00081,G00027,...,UM,OA,994106200802520,L,UM,Top Up,Fresh,STORE00001,LS,E02
578079,200637,20061109,5,16,1,4.47,PRD0900396,CL00229,DEP00081,G00027,...,UM,NaN,994103100229855,L,MM,Full Shop,Mixed,STORE00001,LS,E02
578080,200717,20070624,1,10,1,4.92,PRD0903065,CL00229,DEP00081,G00027,...,UM,YF,994106300437841,L,MM,Full Shop,Mixed,STORE00001,LS,E02


# Data cleaning

In [5]:
df.dropna(subset=['CUST_CODE'], inplace=True)
df['CUST_LIFESTAGE'] = df['CUST_LIFESTAGE'].fillna('XX')
df['SHOP_DATE'] = pd.to_datetime(df['SHOP_DATE'], format='%Y%m%d')
df['SHOP_MONTH'] = df['SHOP_DATE'].dt.to_period('m')
df['BASKET_ID'] = df['BASKET_ID'].astype(str)

SHOP_WEEK                   0
SHOP_DATE                   0
SHOP_WEEKDAY                0
SHOP_HOUR                   0
QUANTITY                    0
SPEND                       0
PROD_CODE                   0
PROD_CODE_10                0
PROD_CODE_20                0
PROD_CODE_30                0
PROD_CODE_40                0
CUST_CODE                   0
CUST_PRICE_SENSITIVITY      0
CUST_LIFESTAGE              0
BASKET_ID                   0
BASKET_SIZE                 0
BASKET_PRICE_SENSITIVITY    0
BASKET_TYPE                 0
BASKET_DOMINANT_MISSION     0
STORE_CODE                  0
STORE_FORMAT                0
STORE_REGION                0
dtype: int64

# Create Customer Single View

In [13]:
# create single customer view
df_scv = df[['CUST_CODE']].drop_duplicates().reset_index(drop=True)

# aggregate for total period
df_scv = df_scv.merge(
    df.groupby(['CUST_CODE', 'BASKET_ID']).agg(
    spend = ('SPEND', 'sum'),
    first_visit = ('SHOP_DATE', 'min'),
    last_visit = ('SHOP_DATE', 'max')).reset_index().groupby('CUST_CODE').agg(avg_bkt_size = ('spend', 'mean'),
                                                                    total_trans = ('spend', 'count'),
                                                                    total_spend = ('spend', 'sum'),
                                                                    first_visit = ('first_visit', 'min'),
                                                                    last_visit = ('last_visit', 'max')),
    how='left', on='CUST_CODE')

# aggregate for 3 months period
df_scv = df_scv.merge(
    df.loc[df['SHOP_MONTH'] > (df['SHOP_MONTH'].max() - 3), :].groupby(['CUST_CODE', 'BASKET_ID']).agg(
    spend = ('SPEND', 'sum')).reset_index().groupby('CUST_CODE').agg(avg_bkt_size_3m = ('spend', 'mean'),
                                                                    total_trans_3m = ('spend', 'count'),
                                                                    total_spend_3m = ('spend', 'sum')),
    how='left', on='CUST_CODE')

# aggregate for 6 months period
df_scv = df_scv.merge(
    df.loc[df['SHOP_MONTH'] > (df['SHOP_MONTH'].max() - 6), :].groupby(['CUST_CODE', 'BASKET_ID']).agg(
    spend = ('SPEND', 'sum')).reset_index().groupby('CUST_CODE').agg(avg_bkt_size_6m = ('spend', 'mean'),
                                                                    total_trans_6m = ('spend', 'count'),
                                                                    total_spend_6m = ('spend', 'sum')),
    how='left', on='CUST_CODE')

df_scv['tbp'] = (df_scv['last_visit']-df_scv['first_visit']).dt.days / df_scv['total_trans']
df_scv['mem_dur'] = (df['SHOP_DATE'].max()-df_scv['first_visit']).dt.days
df_scv['spending_per_day'] = df_scv['total_spend']/df_scv['mem_dur']
df_scv['trans_per_day'] = df_scv['total_trans']/df_scv['mem_dur']


# drop column
df_scv.drop(columns=['first_visit'], inplace=True)

df_scv = df_scv.fillna(0)

display(df_scv)

,CUST_CODE,avg_bkt_size,total_trans,total_spend,last_visit,avg_bkt_size_3m,total_trans_3m,total_spend_3m,avg_bkt_size_6m,total_trans_6m,total_spend_6m,tbp,mem_dur,spending_per_day,trans_per_day
0,CUST0000804514,20.230000,1,20.23,2007-10-19,0.000000,0.0,0.00,0.000000,0.0,0.00,0.000000,261,0.077510,0.003831
1,CUST0000735558,6.150000,2,12.30,2008-03-28,0.000000,0.0,0.00,6.150000,2.0,12.30,1.500000,103,0.119417,0.019417
2,CUST0000138092,9.814601,263,2581.24,2008-07-06,8.880000,23.0,204.24,9.205833,48.0,441.88,3.098859,815,3.167166,0.322699
3,CUST0000099658,17.162606,284,4874.18,2008-07-06,14.859565,23.0,341.77,17.194107,56.0,962.87,2.855634,811,6.010086,0.350185
4,CUST0000673735,21.546667,93,2003.84,2008-06-13,19.168750,8.0,153.35,26.346667,21.0,553.28,8.376344,802,2.498554,0.115960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3434,CUST0000937752,0.750000,1,0.75,2008-01-08,0.000000,0.0,0.00,0.000000,0.0,0.00,0.000000,180,0.004167,0.005556
3435,CUST0000278162,2.000000,1,2.00,2006-04-20,0.000000,0.0,0.00,0.000000,0.0,0.00,0.000000,808,0.002475,0.001238
3436,CUST0000352552,0.630000,1,0.63,2007-11-26,0.000000,0.0,0.00,0.000000,0.0,0.00,0.000000,223,0.002825,0.004484
3437,CUST0000218304,1.540000,1,1.54,2008-01-24,0.000000,0.0,0.00,0.000000,0.0,0.00,0.000000,164,0.009390,0.006098


# Basic Customer Analytics

In [71]:
df_analytics = df_scv.copy()
df_analytics = df_analytics.loc[:,('CUST_CODE','total_trans_3m','total_spend_3m','total_trans_6m','total_spend_6m')]

In [72]:
df_analytics['spending_per_trans_3m'] = df_analytics['total_spend_3m'] / df_analytics['total_trans_3m']

In [73]:
df_analytics['spending_per_trans_6m'] = df_analytics['total_spend_6m'] / df_analytics['total_trans_6m']

In [74]:
df_analytics = df_analytics.fillna(0)

In [75]:
df_analytics = df_analytics = df_analytics[~((df_analytics['spending_per_trans_3m'] == 0) | (df_analytics['spending_per_trans_6m'] == 0))].reset_index(drop = True)

In [80]:
df_analytics

,CUST_CODE,total_trans_3m,total_spend_3m,total_trans_6m,total_spend_6m,spending_per_trans_3m,spending_per_trans_6m,percentile_3m,percentile_6m
0,CUST0000138092,23.0,204.24,48.0,441.88,8.880000,9.205833,47.902501,46.269346
1,CUST0000099658,23.0,341.77,56.0,962.87,14.859565,17.194107,64.161430,69.294883
2,CUST0000673735,8.0,153.35,21.0,553.28,19.168750,26.346667,73.450854,84.361777
3,CUST0000183115,4.0,44.46,8.0,113.64,11.115000,14.205000,55.511464,62.184803
4,CUST0000745448,7.0,53.78,22.0,129.51,7.682857,5.886818,43.813910,33.517796
...,...,...,...,...,...,...,...,...,...
1351,CUST0000691438,1.0,2.38,1.0,2.38,2.380000,2.380000,18.181319,15.385124
1352,CUST0000844929,1.0,1.02,2.0,2.04,1.020000,1.020000,9.098712,8.000000
1353,CUST0000031949,1.0,1.08,1.0,1.08,1.080000,1.080000,9.613734,8.308087
1354,CUST0000950079,1.0,0.51,1.0,0.51,0.510000,0.510000,3.212598,2.750943


In [77]:
# Calculate the percentiles for 'spending_per_trans_3m'
percentile_values_3m = np.percentile(df_analytics['spending_per_trans_3m'], np.arange(0, 101))
df_analytics['percentile_3m'] = df_analytics['spending_per_trans_3m'].apply(lambda x: np.interp(x, percentile_values_3m, np.arange(0, 101)))

# Calculate the percentiles for 'spending_per_trans_6m'
percentile_values_6m = np.percentile(df_analytics['spending_per_trans_6m'], np.arange(0, 101))
df_analytics['percentile_6m'] = df_analytics['spending_per_trans_6m'].apply(lambda x: np.interp(x, percentile_values_6m, np.arange(0, 101)))

# Display the DataFrame
display(df_analytics)

,CUST_CODE,total_trans_3m,total_spend_3m,total_trans_6m,total_spend_6m,spending_per_trans_3m,spending_per_trans_6m,percentile_3m,percentile_6m
0,CUST0000138092,23.0,204.24,48.0,441.88,8.880000,9.205833,47.902501,46.269346
1,CUST0000099658,23.0,341.77,56.0,962.87,14.859565,17.194107,64.161430,69.294883
2,CUST0000673735,8.0,153.35,21.0,553.28,19.168750,26.346667,73.450854,84.361777
3,CUST0000183115,4.0,44.46,8.0,113.64,11.115000,14.205000,55.511464,62.184803
4,CUST0000745448,7.0,53.78,22.0,129.51,7.682857,5.886818,43.813910,33.517796
...,...,...,...,...,...,...,...,...,...
1351,CUST0000691438,1.0,2.38,1.0,2.38,2.380000,2.380000,18.181319,15.385124
1352,CUST0000844929,1.0,1.02,2.0,2.04,1.020000,1.020000,9.098712,8.000000
1353,CUST0000031949,1.0,1.08,1.0,1.08,1.080000,1.080000,9.613734,8.308087
1354,CUST0000950079,1.0,0.51,1.0,0.51,0.510000,0.510000,3.212598,2.750943


In [78]:
df_result = df_analytics[((df_analytics['spending_per_trans_3m'] > df_analytics['spending_per_trans_6m']))&((df_analytics['percentile_3m'] > df_analytics['percentile_6m']))].reset_index(drop = True)

In [79]:
df_result[['CUST_CODE']]

,CUST_CODE
0,CUST0000745448
1,CUST0000122195
2,CUST0000298310
3,CUST0000336781
4,CUST0000167190
...,...
414,CUST0000195177
415,CUST0000058881
416,CUST0000963056
417,CUST0000084563
